In [3]:
# set up
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

path = 'data/melb_data.csv'
data = pd.read_csv(path)

y = data.Price
X = data.drop(['Price'], axis=1)
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [col for col in X_train_full.columns 
if X_train_full[col].nunique()<10 and X_train_full[col].dtype == 'object']

numerical_cols = [col for col in X_train_full.columns
if np.issubdtype(X_train_full[col].dtype, np.number)] 

cols = categorical_cols + numerical_cols
X_train = X_train_full[cols]
X_test = X_test_full[cols]

<img src="static\dtype-hierarchy.png" style="zoom:120%"/>  

the numpy dtype hierarchy img

#### Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to bundle together different preprocessing steps. The code below:
- imputes missing values in numerical data
- imputes missing values and applies a one-hot encoding to categorical data.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ]
)

#### Step2: Define the Model

Next, we define a random forest model with the `RandomForestRegressor` class

In [7]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

#### Step3: Create and Evalute the Pipeline

Finally, we use the Pipeline class to define a pipeline that bundles the preprocessing and modeling steps. There are a few important things to notice:

- With the pipeline, we preprocess the training data and fit the model in a single line of code. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)

- With the pipeline, we supply the unprocessed features in X_valid to the `predict()` command, and the pipeline automatically preprocesses the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)

In [9]:
from sklearn.metrics import mean_absolute_error

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)

mean_absolute_error(y_test, predictions)

160679.18917034855